In [19]:
import requests
import pandas as pd
import math
import json

In [20]:
df = pd.read_csv('data/demo_intermediate.csv', index_col=False)

In [21]:
city_level = df[df['GEO_ID'].str.startswith('16')].to_dict('records')[0] # --> entire new york city estimates and margin of errors

In [22]:
df.loc[~df['GEO_ID'].str.startswith('16'), :].head() # --> all rows except city level 

,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,B01001_005E,B01001_005M,...,DP05_0088E,DP05_0088M,DP05_0088PE,DP05_0088PM,DP05_0089E,DP05_0089M,DP05_0089PE,DP05_0089PM,GEO_ID,NAME
0,32127.0,1246.122386,14961.0,712.721545,694.0,178.555314,937.0,224.926655,1020.0,250.339769,...,9935.0,578.784934,NaN,NaN,11968.0,652.078983,NaN,NaN,BX31,BX31
1,57718.0,1775.077463,27570.0,1175.599422,2768.0,516.036820,1932.0,331.407604,1993.0,323.620148,...,13717.0,916.604604,NaN,NaN,16104.0,830.780958,NaN,NaN,BX05,BX05
2,29990.0,1432.393452,13655.0,964.579701,869.0,237.491052,1232.0,321.619340,1003.0,315.680218,...,7037.0,596.124148,NaN,NaN,8776.0,631.763405,NaN,NaN,BX06,BX06
3,38083.0,1482.243570,18710.0,1032.483898,1660.0,380.988189,1240.0,297.465964,1169.0,288.263768,...,10555.0,741.169346,NaN,NaN,12806.0,706.343401,NaN,NaN,BX07,BX07
4,35856.0,1580.290163,15986.0,1231.871341,1703.0,411.582313,1559.0,310.049996,1327.0,271.239746,...,8390.0,1023.547263,NaN,NaN,11047.0,687.457635,NaN,NaN,BX01,BX01


### Formula:
https://www.census.gov/content/dam/Census/library/publications/2018/acs/acs_general_handbook_2018_ch08.pdf

In [23]:
# e --> estimate, m --> moe
def get_c(e, m): 
    if str(m).strip():
        return ''
    elif e == 0:
        return ''
    else: 
        return m/1.645/e*100

def get_p(e, agg_e):
    if agg_e == 0: 
        return ''
    else: 
        return e/agg_e*100

def get_z(e, m, p, agg_e, agg_m):
    if p == 0:
        return ''
    elif p == 100:
        return ''
    elif agg_e == 0:
        return ''
    elif m**2 - (e*agg_m/agg_e)**2 <0:
        return math.sqrt(m**2 + (e*agg_m/agg_e)**2)/agg_e*100
    else: 
        return math.sqrt(m**2 - (e*agg_m/agg_e)**2)/agg_e*100

In [24]:
%%time
variables = list(df.columns)
variables.remove('GEO_ID')
variables.remove('NAME')
var = list(set([i[:-1] for i in variables if i[:-1][-1] != 'P']))
for i in var:
    df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}C'] = df.apply(lambda row: get_c(row[f'{i}E'], 
                                                                                    row[f'{i}M']), axis=1)
    df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.apply(lambda row: get_p(row[f'{i}E'], 
                                                                  city_level[f'{i}E']), axis=1)
    df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}Z'] = df.apply(lambda row: get_z(row[f'{i}E'], 
                                                                    row[f'{i}M'], 
                                                                    row[f'{i}P'],
                                                                    city_level[f'{i}E'],
                                                                    city_level[f'{i}M']), axis=1)
    # for i in var:
#     df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}C'] = df.apply(lambda row: get_c(row[f'{i}E'], row[f'{i}M']), axis=1)
    
#     if f'{i}PE' in list(df.columns):
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}PE']
#     else: 
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.apply(lambda row: get_p(row[f'{i}E'], 
#                                                                                                 city_level[f'{i}E']), axis=1)
#     if f'{i}PM' in list(df.columns):
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}P'] = df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}PM']
#     else: 
#         df.loc[~df['GEO_ID'].str.startswith('16'),f'{i}Z'] = df.apply(lambda row: get_z(row[f'{i}E'], 
#                                                                                                 row[f'{i}M'], 
#                                                                                                 row[f'{i}P'], 
#                                                                                                 city_level[f'{i}E'], 
#                                                                                                 city_level[f'{i}M']), axis=1)

CPU times: user 4min 33s, sys: 5.34 s, total: 4min 38s
Wall time: 4min 43s


In [25]:
output_cols = [[i+'E', i+'M', i+'P', i+'Z'] for i in var]

In [26]:
output = ['GEO_ID', 'NAME']
for i in output_cols:
    output+=i

In [27]:
df.columns

Index(['B01001_001E', 'B01001_001M', 'B01001_002E', 'B01001_002M',
       'B01001_003E', 'B01001_003M', 'B01001_004E', 'B01001_004M',
       'B01001_005E', 'B01001_005M',
       ...
       'B01001_003Z', 'DP05_0001C', 'DP05_0001P', 'DP05_0001Z', 'DP05_0008C',
       'DP05_0008P', 'DP05_0008Z', 'B01001_001C', 'B01001_001P',
       'B01001_001Z'],
      dtype='object', length=1265)

In [28]:
df[output].head()

,GEO_ID,NAME,DP05_0020E,DP05_0020M,DP05_0020P,DP05_0020Z,DP05_0045E,DP05_0045M,DP05_0045P,DP05_0045Z,...,DP05_0001P,DP05_0001Z,DP05_0008E,DP05_0008M,DP05_0008P,DP05_0008Z,B01001_001E,B01001_001M,B01001_001P,B01001_001Z
0,BX31,BX31,26569.0,993.438976,0.382045,0.0142847,301.0,117.089709,0.130532,0.0507013,...,0.375312,24.3581,2309.0,311.244277,0.490490,0.0661155,32127.0,1246.122386,0.375312,24.3581
1,BX05,BX05,43498.0,1397.755343,0.625473,0.0200982,666.0,264.981131,0.288819,0.114748,...,0.674270,43.7607,4052.0,518.311682,0.860747,0.110101,57718.0,1775.077463,0.674270,43.7607
2,BX06,BX06,22829.0,1052.337873,0.328266,0.0151317,123.0,146.451357,0.053341,0.0635003,...,0.350348,22.7378,4062.0,507.461329,0.862871,0.107796,29990.0,1432.393452,0.350348,22.7378
3,BX07,BX07,30186.0,1162.177697,0.434055,0.016711,412.0,177.679487,0.178669,0.076959,...,0.444891,28.8738,2325.0,338.912968,0.493889,0.0719931,38083.0,1482.243570,0.444891,28.8738
4,BX01,BX01,25072.0,1248.525130,0.360519,0.0179528,27.0,45.409250,0.011709,0.0196907,...,0.418875,27.1853,2990.0,450.833672,0.635151,0.0957677,35856.0,1580.290163,0.418875,27.1853


In [29]:
df[['B01001_001E', 'B01001_001M', 'B01001_001C', 'B01001_001P', 'B01001_001Z']].head()

,B01001_001E,B01001_001M,B01001_001C,B01001_001P,B01001_001Z
0,32127.0,1246.122386,,0.375312,24.3581
1,57718.0,1775.077463,,0.674270,43.7607
2,29990.0,1432.393452,,0.350348,22.7378
3,38083.0,1482.243570,,0.444891,28.8738
4,35856.0,1580.290163,,0.418875,27.1853


#### Notes: 
for this following group, we do have e.g. Percent Margin of Error, maybe we shouldn't calculate these
```json
"DP05_0089PM": {
    "label": "Percent Margin of Error!!CITIZEN VOTING AGE POPULATION!!Citizen 18 and over population!!Female",
    "concept": "ACS DEMOGRAPHIC AND HOUSING ESTIMATES",
    "predicateType": "float",
    "group": "DP05",
    "limit": 0,
    "predicateOnly": true
},
```
https://api.census.gov/data/2017/acs/acs5/profile/groups/DP05.json